In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
from crims2s.distribution import fit_gamma_xarray
from crims2s.mldataset import read_flat_fields, make_model_params
from crims2s.util import add_biweekly_dim

In [ ]:
INPUT_DIR = '***BASEDIR***/training-input/'

In [ ]:
eccc_data = read_flat_fields(INPUT_DIR, "eccc", ["t2m", "tp"], '20200102', file_label='hindcast')
eccc_data = (
    eccc_data.isel(forecast_monthday=0)
    .sel(forecast_year=2000)
    .transpose("lead_time", "latitude", "longitude", "realization")
)

In [ ]:
eccc_data

In [ ]:
eccc_data.valid_time.compute()

In [ ]:
eccc_data.tp.interpolate_na(dim='realization').compute()

In [ ]:
ds = make_model_params(eccc_data, interp_tp_na=True, )

In [ ]:
ds.tp_cube_root_sigma.isel(biweekly_forecast=0).plot()

In [ ]:
model_biweekly = add_biweekly_dim(eccc_data, weeks_12=False)

In [ ]:
max_valid_time = model_biweekly.forecast_time + pd.to_timedelta('32D')

In [ ]:
max_leads = model_biweekly.valid_time.where(model_biweekly.valid_time < max_valid_time, model_biweekly.forecast_time).argmax(dim='lead_time')

In [ ]:
max_leads

In [ ]:
model_biweekly.valid_time

In [ ]:
nointerp = model_biweekly.isel(lead_time=max_leads).tp.isel(biweekly_forecast=1, realization=0)
interp = model_biweekly.isel(lead_time=max_leads).tp.isel(biweekly_forecast=1, realization=0).interpolate_na(dim='longitude')

diff = nointerp != interp

In [ ]:
interp.where(diff).mean().std().compute()

In [ ]:
interp_model_biweekly = model_biweekly.isel(lead_time=max_leads)
for max_gap in [1, 5, 10]:
    interp_model_biweekly = model_biweekly.interpolate_na(dim='longitude', max_gap=max_gap, use_coordinate=False).interpolate_na(dim='latitude', max_gap=max_gap, use_coordinate=False)

In [ ]:
interp_model_biweekly.isel(lead_time=max_leads).tp.isel(realization=1, biweekly_forecast=0).isnull().plot()

In [ ]:
interp_model_biweekly.isel(lead_time=max_leads).isnull().any(dim=['latitude', 'longitude']).compute()

In [ ]:
fit_gamma_xarray(interp_model_biweekly.isel(lead_time=max_leads).tp, dim='realization')